In [11]:
# %load poi_id.py
#!/usr/bin/python

import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list = ['poi', 'salary']
#features_list = ['poi'
#                 ,'salary'
#                 , 'deferral_payments'
#                 , 'total_payments'
#                 , 'loan_advances'
#                 , 'bonus'
#                 , 'bonus_salary_ratio'
#                 , 'restricted_stock_deffered'
#                 , 'deferred_income'
#                 , 'total_stock_value'
#                 , 'expenses'
#                 , 'exercised_stock_options'
#                 , 'other'
#                 , 'long_term_incentive'
#                 , 'restricted_stock'
#                 , 'director_fees'
#                 , 'to_messages'
#                 , 'from_poi_to_this_person'
#                 , 'from_poi_to_this_person_percentage'
#                 , 'from_messages'
#                 , 'from_this_person_to_poi'
#                 , 'from_this_person_to_poi_percentage'
#                 , 'shared_receipt_with_poi'
#                 ] # You will need to use more features

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

### Task 2: Remove outliers
#data_dict.pop('Total')
#data_dict.pop('THE TRAVEL AGENCY IN THE PARK')

### Task 3: Create new feature(s)
### Store to my_dataset for easy export below.
my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)

In [14]:
import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".

features_list = [
                 'poi',
                 'salary',
                 # 'deferral_payments',
                 # 'total_payments',
                 # 'loan_advances',
                 'bonus',
                 'bonus_salary_ratio',
                 # 'restricted_stock_deferred',
                 # 'deferred_income',
                 'total_stock_value',
                 # 'expenses',
                 'exercised_stock_options',
                 # 'other',
                 # 'long_term_incentive',
                 # 'restricted_stock',
                 # 'director_fees',
                 # 'to_messages',
                 # 'from_poi_to_this_person',
                 # 'from_poi_to_this_person_percentage',
                 # 'from_messages',
                 # 'from_this_person_to_poi',
                 'from_this_person_to_poi_percentage',
                 'shared_receipt_with_poi'
                 ]

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

### Task 2: Remove outliers

data_dict.pop("TOTAL")
data_dict.pop("THE TRAVEL AGENCY IN THE PARK")

### Task 3: Create new feature(s)

# Bonus-salary ratio
for employee, features in data_dict.iteritems():
	if features['bonus'] == "NaN" or features['salary'] == "NaN":
		features['bonus_salary_ratio'] = "NaN"
	else:
		features['bonus_salary_ratio'] = float(features['bonus']) / float(features['salary'])

# from_this_person_to_poi as a percentage of from_messages
for employee, features in data_dict.iteritems():
	if features['from_this_person_to_poi'] == "NaN" or features['from_messages'] == "NaN":
		features['from_this_person_to_poi_percentage'] = "NaN"
	else:
		features['from_this_person_to_poi_percentage'] = float(features['from_this_person_to_poi']) / float(features['from_messages'])

# from_poi_to_this_person as a percentage of to_messages
for employee, features in data_dict.iteritems():
	if features['from_poi_to_this_person'] == "NaN" or features['to_messages'] == "NaN":
		features['from_poi_to_this_person_percentage'] = "NaN"
	else:
		features['from_poi_to_this_person_percentage'] = float(features['from_poi_to_this_person']) / float(features['to_messages'])

### Impute missing email features to mean
email_features = ['to_messages',
	              'from_poi_to_this_person',
	              'from_poi_to_this_person_percentage',
	              'from_messages',
	              'from_this_person_to_poi',
	              'from_this_person_to_poi_percentage',
	              'shared_receipt_with_poi']
from collections import defaultdict
email_feature_sums = defaultdict(lambda:0)
email_feature_counts = defaultdict(lambda:0)

for employee, features in data_dict.iteritems():
	for ef in email_features:
		if features[ef] != "NaN":
			email_feature_sums[ef] += features[ef]
			email_feature_counts[ef] += 1

email_feature_means = {}
for ef in email_features:
	email_feature_means[ef] = float(email_feature_sums[ef]) / float(email_feature_counts[ef])

for employee, features in data_dict.iteritems():
	for ef in email_features:
		if features[ef] == "NaN":
			features[ef] = email_feature_means[ef]

### Store to my_dataset for easy export below.
my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys=True)
labels, features = targetFeatureSplit(data)

### Task 4: Try a variety of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

# Potential pipeline steps
scaler = MinMaxScaler()
select = SelectKBest()
dtc = DecisionTreeClassifier()
svc = SVC()
knc = KNeighborsClassifier()

# Load pipeline steps into list
steps = [
		 # Preprocessing
         # ('min_max_scaler', scaler),
         
         # Feature selection
         ('feature_selection', select),
         
         # Classifier
         ('dtc', dtc)
         # ('svc', svc)
         # ('knc', knc)
         ]

# Create pipeline
pipeline = Pipeline(steps)

# Parameters to try in grid search
parameters = dict(
                  feature_selection__k=[2, 3, 5, 6], 
                  dtc__criterion=['gini', 'entropy'],
                  # dtc__splitter=['best', 'random'],
                  dtc__max_depth=[None, 1, 2, 3, 4],
                  dtc__min_samples_split=[1, 2, 3, 4, 25],
                  # dtc__min_samples_leaf=[1, 2, 3, 4],
                  # dtc__min_weight_fraction_leaf=[0, 0.25, 0.5],
                  dtc__class_weight=[None, 'balanced'],
                  dtc__random_state=[42]
                  # svc__C=[0.1, 1, 10, 100, 1000],
                  # svc__kernel=['rbf'],
                  # svc__gamma=[0.001, 0.0001]
                  # knc__n_neighbors=[1, 2, 3, 4, 5],
                  # knc__leaf_size=[1, 10, 30, 60],
                  # knc__algorithm=['auto', 'ball_tree', 'kd_tree', 'brute']
                  )

### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

from sklearn.cross_validation import train_test_split, StratifiedShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

# Create training sets and test sets
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

# Cross-validation for parameter tuning in grid search 
sss = StratifiedShuffleSplit(
    labels_train,
    n_iter = 20,
    test_size = 0.5,
    random_state = 0
    )

# Create, fit, and make predictions with grid search
gs = GridSearchCV(pipeline,
	              param_grid=parameters,
	              scoring="f1",
	              cv=sss,
	              error_score=0)
gs.fit(features_train, labels_train)
labels_predictions = gs.predict(features_test)

# Pick the classifier with the best tuned parameters
clf = gs.best_estimator_
print "\n", "Best parameters are: ", gs.best_params_, "\n"

# Print features selected and their importances
features_selected=[features_list[i+1] for i in clf.named_steps['feature_selection'].get_support(indices=True)]
scores = clf.named_steps['feature_selection'].scores_
importances = clf.named_steps['dtc'].feature_importances_
import numpy as np
indices = np.argsort(importances)[::-1]
print 'The ', len(features_selected), " features selected and their importances:"
for i in range(len(features_selected)):
    print "feature no. {}: {} ({}) ({})".format(i+1,features_selected[indices[i]],importances[indices[i]], scores[indices[i]])

# Print classification report (focus on precision and recall)
report = classification_report( labels_test, labels_predictions )
print(report)

### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)


Best parameters are:  {'dtc__max_depth': 2, 'dtc__criterion': 'gini', 'dtc__min_samples_split': 2, 'dtc__class_weight': 'balanced', 'feature_selection__k': 6, 'dtc__random_state': 42} 

The  6  features selected and their importances:
feature no. 1: bonus_salary_ratio (0.658595952706) (22.1067164085)
feature no. 2: shared_receipt_with_poi (0.180270198721) (6.1299573021)
feature no. 3: total_stock_value (0.161133848573) (16.8651432616)
feature no. 4: exercised_stock_options (0.0) (16.9328653375)
feature no. 5: bonus (0.0) (34.2129648303)
feature no. 6: salary (0.0) (17.7678544529)
             precision    recall  f1-score   support

        0.0       0.96      0.69      0.81        39
        1.0       0.25      0.80      0.38         5

avg / total       0.88      0.70      0.76        44



In [15]:
print 'done'

done


In [16]:
#!/usr/bin/python

import sys
import pickle
import json
import numpy as np
import time
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data, test_classifier

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import MaxAbsScaler

from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid

from sklearn.cross_validation import StratifiedShuffleSplit

import warnings


# suppress warnings
warnings.filterwarnings('ignore')


# There was one user (EUGENE) with no data
def find_users_with_no_data(scan_data):
    no_data_users = []
    for ppl, features in scan_data.items():
        non_NaN = False
        for feature, value in features.items():
            if feature != 'poi':
                if value != 'NaN':
                    non_NaN = True
                    break

        if not non_NaN:
            no_data_users.append(ppl)

    return no_data_users


# There are no features with NaN set as the value for every user
def find_features_with_no_data(scan_data, feature_list):
    no_data_features = []
    for feature in feature_list:
        non_NaN = False
        for ppl, features in scan_data.items():
            try:
                if features[feature] != 'NaN':
                    non_NaN = True
                    break
            except KeyError:
                pass

        if not non_NaN:
            no_data_features.append(feature)

    return no_data_features


# Simple function to print out all the data from the data_dict
def display_values(data):
    # fields_of_interest = ['poi', 'deferral_payments', 'restricted_stock_deferred', 'total_stock_value', 'restricted_stock']
    fields_of_interest = ['restricted_stock_deferred']
    # persons_of_interest = ['BELFER ROBERT','BHATNAGAR SANJAY']
    persons_of_interest = []
    if len(persons_of_interest) > 0:
        for person in persons_of_interest:
            print(person, json.dumps(data[person], indent=4))
    else:
        for person, values in data.items():
            poi = data[person]['poi']
            for key, value in values.items():
                if key in fields_of_interest and value != 'NaN':
                    print(person, poi, key, value)


# Function to correct the entries in the data dictionary that were incorrectly set originally
def correct_invalid_values(data_to_correct):
    # There are some data input errors for BELFER ROBERT and BHATNAGAR SANJAY
    # This function corrects those errors back to their original state as per the provided PDF
    empty_value = 'NaN'

    data_to_correct['BELFER ROBERT']['deferred_income'] = -102500
    data_to_correct['BELFER ROBERT']['deferral_payments'] = empty_value
    data_to_correct['BELFER ROBERT']['expenses'] = 3285
    data_to_correct['BELFER ROBERT']['director_fees'] = 102500
    data_to_correct['BELFER ROBERT']['total_payments'] = 3285
    data_to_correct['BELFER ROBERT']['exercised_stock_options'] = empty_value
    data_to_correct['BELFER ROBERT']['restricted_stock'] = 44093
    data_to_correct['BELFER ROBERT']['restricted_stock_deferred'] = -44093
    data_to_correct['BELFER ROBERT']['total_stock_value'] = empty_value

    data_to_correct['BHATNAGAR SANJAY']['other'] = empty_value
    data_to_correct['BHATNAGAR SANJAY']['expenses'] = 137864
    data_to_correct['BHATNAGAR SANJAY']['director_fees'] = empty_value
    data_to_correct['BHATNAGAR SANJAY']['total_payments'] = 137864
    data_to_correct['BHATNAGAR SANJAY']['exercised_stock_options'] = 15456290
    data_to_correct['BHATNAGAR SANJAY']['restricted_stock'] = 2604490
    data_to_correct['BHATNAGAR SANJAY']['restricted_stock_deferred'] = -2604490
    data_to_correct['BHATNAGAR SANJAY']['total_stock_value'] = 15456290

    return data_to_correct


# Simple function to count how many non-zero values there are for each feature
def count_zeros_and_non_zeros(column):
    zeros = 0
    non_zeros = 0
    for entry in column:
        if entry == 0:
            zeros += 1
        else:
            non_zeros += 1

    return zeros, non_zeros


# Create some new features that show the % of emails sent and received by each individual to/from POIs
# These numbers may be more predictive than just absolute numbers as all individuals send different amounts of email
def create_new_features(existing_data):
    for person, features in existing_data.items():
        existing_data[person]['from_poi_pct'] = create_pct(features['from_poi_to_this_person'], features['to_messages'])
        existing_data[person]['to_poi_pct'] = create_pct(features['from_this_person_to_poi'], features['from_messages'])

    return existing_data


# Function to turn a number into a percent of a total value; used for feature engineering
def create_pct(partial, total):
    # Make sure the data is not NaN (if so return 0)
    if partial == 'NaN':
        partial = 0
    if total == 'NaN':
        return 0

    # If neither value is NAN, then return a percentage as a float value
    return (float(partial) / float(total)) * 100.


# Simple function to break the features and labels out of the original data dictionary
def split_features_from_labels(raw_data, target_features):
    new_data = featureFormat(raw_data, target_features, sort_keys=True)
    return targetFeatureSplit(new_data)


# Use the feature and labels data to try all possible combinations of parameters
# Return the best estimator (if it is performing above the baseline, and better than previous entries
# Otherwise return the previous estimator and score
def find_best_estimator_for_grid(classifier, pipe, params, data_set, feature_list, previous_score, previous_estimator):
    print('')
    print('-------------------------------------')
    print('--------- {classifier}'.format(classifier=classifier))
    print('-------------------------------------')
    print('')

    start = time.time()
    print('Started at {time}'.format(time=time.strftime("%H:%M:%S")))
    print('')

    np.random.seed(0)

    labels, features = split_features_from_labels(data_set, feature_list)

    cv = StratifiedShuffleSplit(labels, random_state=42)

    grid = GridSearchCV(pipe, param_grid=params, cv=cv, scoring='f1', n_jobs=1, verbose=1)
    grid.fit(features, labels)

    best_estimator = grid.best_estimator_

    # modified tester returns the scores for comparison purposes
    precision, recall, f1 = test_classifier(grid.best_estimator_, data_set, feature_list)
    if grid.best_params_.get('reduce_dim__k'):
        print('{x} features selected by SelectKBest'.format(x=grid.best_params_['reduce_dim__k']))
        for idx, feature in enumerate(feature_list[1:]):
            print(
                feature,
                grid.best_params_['reduce_dim'].scores_[idx],
                grid.best_params_['reduce_dim'].get_support()[idx]
            )
    else:
        print('PCA used for dim reduction; no features reportable.')

    # return the previous score and estimator if this is not better
    new_score = previous_score
    new_estimator = previous_estimator

    if precision >= 0.3 and recall >= 0.3 and f1 > previous_score:
        new_score = f1
        new_estimator = best_estimator

    print('-------------------------------------')
    end = time.time()
    log_time_to_complete(classifier, start, end)

    return new_estimator, new_score


# Simple logger to track how long each classifier is taking to analyze
def log_time_to_complete(classifier, start_time, end_time):
    print('Took {time} seconds, to find best {classifier}'.format(classifier=classifier, time=end_time-start_time))


# Pulling in all features, will select best features automatically in pipeline (either PCA or SelectKBest)
features_list = [
    'poi',
    'salary', 'to_messages', 'deferral_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred',
    'deferred_income', 'total_stock_value', 'expenses', 'from_poi_to_this_person', 'exercised_stock_options',
    'from_messages', 'other','from_this_person_to_poi', 'long_term_incentive', 'shared_receipt_with_poi',
    'restricted_stock', 'director_fees'
]

# Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "rb") as data_file:
    data_dict = pickle.load(data_file)

####
# Previous validation checks done to fix data
# outlier exists for "TOTAL" row of data
####

# print(sorted(data_dict))
# print(find_users_with_no_data(data_dict))
# print(find_features_with_no_data(data_dict, features_list[1:]))

####
# Remove the invalid outliers completely
####
outliers = ['TOTAL', 'LOCKHART EUGENE E', 'THE TRAVEL AGENCY IN THE PARK']
for outlier in outliers:
    data_dict.pop(outlier, 0)

####
# Fix the erroneous data for 2 entries
####
# display_values(data_dict)
data_dict = correct_invalid_values(data_dict)

# Task 3: Create new feature(s)
data_dict = create_new_features(data_dict)

# Store to my_dataset for easy export below.
my_dataset = data_dict

# Extract features and labels from dataset for local testing
labels, features = split_features_from_labels(my_dataset, features_list)
best_estimator = None
best_score = 0.

# Hyperparameter tuning tests
# General parameters (all classifiers)
n_features = range(2, 19)
scalers = [None, MaxAbsScaler()]

# KNearestNeighbor parameters
n_neighbours = [1, 3, 5, 7, 9, 10, 15, 20, 50]
n_neighbor_weights = ['uniform','distance']
n_neighbor_p = [1, 2]

# Centroid parameters
centroid_distances = ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan']
centroid_shrink = [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# DecisionTree parameters
tree_criterion = ['entropy', 'gini']
tree_splitter = ['random', 'best']
tree_min_splits = range(2, 20, 2)
tree_max_features = ['sqrt', 'log2', None]

# RandomForest parameters
forest_estimators = [5, 10, 15, 20, 25]

# AdaBoost parameters
boost_estimators = [25, 50, 75, 100, 500]

####
# Now try each different type of classifier to see how they perform independently
# Try out Gaussian NB
####
pipe = Pipeline([
    ('scale', MaxAbsScaler()),
    ('reduce_dim', PCA(random_state=42)),
    ('clf', GaussianNB())
])
param_grid = [
    {
        'scale': scalers,
        'reduce_dim': [PCA(random_state=42)],
        'reduce_dim__n_components': n_features
    },
    {
        'scale': scalers,
        'reduce_dim': [SelectKBest()],
        'reduce_dim__k': n_features
    }
]

best_estimator, best_score = find_best_estimator_for_grid('GaussianNB', pipe, param_grid, my_dataset, features_list, best_score, best_estimator)

####
# Now try KNearestNeighbors
####
pipe = Pipeline([
    ('scale', MaxAbsScaler()),
    ('reduce_dim', PCA(random_state=42)),
    ('clf', KNeighborsClassifier())
])
param_grid = [
    {
        'scale': scalers,
        'reduce_dim': [PCA(random_state=42)],
        'reduce_dim__n_components': n_features,
        'clf__n_neighbors': n_neighbours,
        'clf__weights': n_neighbor_weights,
        'clf__p': n_neighbor_p
    },
    {
        'scale': scalers,
        'reduce_dim': [SelectKBest()],
        'reduce_dim__k': n_features,
        'clf__n_neighbors': n_neighbours,
        'clf__weights': n_neighbor_weights,
        'clf__p': n_neighbor_p
    }
]

best_estimator, best_score = find_best_estimator_for_grid('KNearestNeighbors', pipe, param_grid, my_dataset, features_list, best_score, best_estimator)

####
#  Now try NearestCentroid
####
pipe = Pipeline([
    ('scale', MaxAbsScaler()),
    ('reduce_dim', PCA(random_state=42)),
    ('clf', NearestCentroid())
])
param_grid = [
    {
        'scale': scalers,
        'reduce_dim': [PCA(random_state=42)],
        'reduce_dim__n_components': n_features,
        'clf__metric': centroid_distances,
        'clf__shrink_threshold': centroid_shrink
    },
    {
        'scale': scalers,
        'reduce_dim': [SelectKBest()],
        'reduce_dim__k': n_features,
        'clf__metric': centroid_distances,
        'clf__shrink_threshold': centroid_shrink
    }
]

best_estimator, best_score = find_best_estimator_for_grid('NearestCentroid', pipe, param_grid, my_dataset, features_list, best_score, best_estimator)


####
#  Now try DecisionTree
####
pipe = Pipeline([
    ('reduce_dim', PCA(random_state=42)),
    ('clf', DecisionTreeClassifier(random_state=42))
])
param_grid = [
    {
        'reduce_dim': [PCA(random_state=42)],
        'reduce_dim__n_components': n_features,
        'clf__criterion': tree_criterion,
        'clf__max_features': tree_max_features,
        'clf__splitter': tree_splitter,
        'clf__min_samples_split': tree_min_splits
    },
    {
        'reduce_dim': [SelectKBest()],
        'reduce_dim__k': n_features,
        'clf__criterion': tree_criterion,
        'clf__max_features': tree_max_features,
        'clf__splitter': tree_splitter,
        'clf__min_samples_split': tree_min_splits
    }
]

best_estimator, best_score = find_best_estimator_for_grid('DecisionTree', pipe, param_grid, my_dataset, features_list, best_score, best_estimator)


####
#  Now try RandomForest
####
pipe = Pipeline([
    ('reduce_dim', PCA(random_state=42)),
    ('clf', RandomForestClassifier(random_state=42))
])
param_grid = [
    {
        'reduce_dim': [PCA(random_state=42)],
        'reduce_dim__n_components': n_features,
        'clf__n_estimators': forest_estimators,
        'clf__max_features': tree_max_features,
        'clf__min_samples_split': tree_min_splits
    },
    {
        'reduce_dim': [SelectKBest()],
        'reduce_dim__k': n_features,
        'clf__n_estimators': forest_estimators,
        'clf__max_features': tree_max_features,
        'clf__min_samples_split': tree_min_splits
    }
]

# best_estimator, best_score = find_best_estimator_for_grid('RandomForest', pipe, param_grid, my_dataset, features_list, best_score, best_estimator)

####
#  Now try AdaBoost
####
pipe = Pipeline([
    ('reduce_dim', PCA(random_state=42)),
    ('clf', AdaBoostClassifier(random_state=42))
])
param_grid = [
    {
        'reduce_dim': [PCA(random_state=42)],
        'reduce_dim__n_components': n_features,
        'clf__n_estimators': boost_estimators
    },
    {
        'reduce_dim': [SelectKBest()],
        'reduce_dim__k': n_features,
        'clf__n_estimators': boost_estimators
    }
]

# best_estimator, best_score = find_best_estimator_for_grid('AdaBoost', pipe, param_grid, my_dataset, features_list, best_score, best_estimator)

####
# Write out the details of the absolutely best estimator and the score received (f1 value)
####
print(best_score, best_estimator)

####
# Now that we know what classifier is most effective, persist it to disk for testing
####
dump_classifier_and_data(best_estimator, my_dataset, features_list)


-------------------------------------
--------- GaussianNB
-------------------------------------

Started at 20:09:05

Fitting 10 folds for each of 68 candidates, totalling 680 fits


[Parallel(n_jobs=1)]: Done 680 out of 680 | elapsed:    4.6s finished


Pipeline(steps=[('scale', MaxAbsScaler(copy=True)), ('reduce_dim', PCA(copy=True, iterated_power='auto', n_components=7, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', GaussianNB(priors=None))])
	Accuracy: 0.81320	Precision: 0.31606	Recall: 0.34450	F1: 0.32967	F2: 0.33841
	Total predictions: 15000	True positives:  689	False positives: 1491	False negatives: 1311	True negatives: 11509



TypeError: 'NoneType' object is not iterable

In [17]:
import sys
sys.path.append("../ud120-projects/tools/")
import pickle
import collections
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from time import time
import pprint
from feature_format import featureFormat, targetFeatureSplit
from tester import test_classifier, dump_classifier_and_data
from wk_functions import check_for_outliers_95, check_for_outliers_name, check_for_outliers_NaNs, xval_classifier, best_features, tune_RandomForest, tune_AdaBoost, line
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.cross_validation import train_test_split, StratifiedShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_classif

import warnings
warnings.filterwarnings("ignore")

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".

##Shortened list of features to important ones based on findings from previously running the script
poi_label = ['poi']
#financial_features = ['salary', 'total_payments', 'loan_advances', 'bonus',
#                      'deferred_income', 'total_stock_value',
#                      'expenses', 'exercised_stock_options', 'long_term_incentive',
#                      'restricted_stock']
                      # Removed other
#email_features = ['from_poi_to_this_person', 'shared_receipt_with_poi'] 
                  # Removed email_address
				  
financial_features = ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus',
                      'restricted_stock_deferred', 'deferred_income', 'total_stock_value',
                      'expenses', 'exercised_stock_options', 'long_term_incentive',
                      'restricted_stock', 'director_fees']
email_features = ['to_messages', 'from_poi_to_this_person', 'from_messages',
                  'from_this_person_to_poi', 'shared_receipt_with_poi']

#financial_features = ['salary', 'deferral_payments']
#email_features = ['to_messages', 'from_poi_to_this_person']

line()
features_list = poi_label + financial_features + email_features
print('Total Starting Features:', len(features_list))

### Load the dictionary containing the dataset
with open("final_project/final_project_dataset.pkl", "rb") as data_file:
    data_dict = pickle.load(data_file)
    #data_dict = pd.DataFrame(data_dict)
print('Total Data Points:', len(data_dict))

NaNs = [0 for i in range(len(features_list))]
for i, person in enumerate(data_dict.values()):
    for j, feature in enumerate(features_list):
        if person[feature] == 'NaN':
            NaNs[j] += 1
line() # Check for NaNs in each feature
print('# NaNs in Each Feature:')
for i, feature in enumerate(features_list):
    print(feature + ': ', NaNs[i])

### Task 2: Remove outliers
line()
print('95% Percentile Financial Feature Values')
line()
print('salary/bonus:')
check_for_outliers_95('bonus', 'salary', data_dict)
    #   LAY KENNETH L 7000000 1072321
    #   SKILLING JEFFREY K 5600000 1111258
    #   TOTAL 97343619 26704229
line()
print('exercised stock options/long term incentive:')
check_for_outliers_95('exercised_stock_options', 'long_term_incentive', data_dict)
    #   LAY KENNETH L 34348384 3600000
    #   TOTAL 311764000 48521928
line()
print('TOTAL is an outlier that sums the rest of the data')
print('It should be removed from the dataset')
data_dict.pop('TOTAL')

line()
print('Check employee names:')
check_for_outliers_name(data_dict)
line()
print('THE TRAVEL AGENCY IN THE PARK is not an employee, and should be removed from the list')
data_dict.pop('THE TRAVEL AGENCY IN THE PARK')

line()
check_for_outliers_NaNs(data_dict, features_list)
print('LOCKHART EUGENE E has only NaN values, so he should be removed from the list')
data_dict.pop('LOCKHART EUGENE E')

line()
print('Updated Total Data Points:', len(data_dict))
line()

# Check number of persons of interest
npoi = 0
for p in data_dict.values():
    if p['poi']:
        npoi += 1
print ("Total POIs: ", npoi)
print ("Total Non-POIs: ", len(data_dict) - npoi)
line()

### Task 3: Create new feature(s)

### Store to my_dataset for easy export below.
my_dataset = data_dict

#### Comment out below section to score classifiers without the new features
## Create new features: to_poi_ratio and from_poi_ratio
for person in my_dataset.values():
    person['to_poi_ratio'] = 0
    person['from_poi_ratio'] = 0
    person['blank'] = 0
    if float(person['from_messages']) > 0:
        person['to_poi_ratio'] = \
            float(person['from_this_person_to_poi']) / float(person['from_messages'])
    if float(person['from_messages']) > 0:
        person['from_poi_ratio'] = \
            float(person['from_poi_to_this_person']) / float(person['to_messages'])

features_list.extend(['to_poi_ratio', 'from_poi_ratio', 'blank'])
print(features_list)
print('New features created: to_poi_ratio, from_poi_ratio')
print('These features calculate the ratio of emails to or from POIs out')
print('of total sent or received')
line()
print('to_poi_ratio: from_this_person_to_poi / from_messages')
print('from_poi_ratio: from_poi_to_this_person / to_messages')
print('Features list was updated with to_poi_ratio')
print('Updated Total Features:', len(features_list) - 1)
line()
#### End comment section

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

## Use StratifiedShuffleSplit() to try to identify best features for the model
sss = StratifiedShuffleSplit(labels, 1000, random_state = 666)
print(sss)
print('Split Data Set Using StratifiedShuffleSplit for best feature identification')

line()    

### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.

# Assign variables for classifier comparisons, these were taken from the suggestions in
# the choose_your_own algorithm project: AdaBoost and Random Forest

print('Begin training AdaBoost and RandomForest Classifiers...')
print('Please wait...')
print('Training {0} Features...'.format(len(features_list) - 1))
line()

scores = []
ada_accuracy = []	
ada_precision = []
ada_recall = []
rf_accuracy = []
rf_precision = []
rf_recall = []

for i in range(len(features[0])):
    t = time()
    sel = SelectKBest(f_classif, k = i + 1)
    sel.fit(features, labels)
    reduced_features = sel.fit_transform(features, labels)
    stop = np.sort(sel.scores_)[::-1][i]
    selected_features_list = [f for j, f in enumerate(features_list[1:]) if sel.scores_[j] >= stop]
    selected_features_list = ['poi'] + selected_features_list
    
    # Run metrics on AdaBoost Classifier
    ada = AdaBoostClassifier(random_state = 666)
    acc, prec, re = xval_classifier(ada, reduced_features, labels, sss)
    ada_accuracy.append(round(float(acc), 2))
    ada_precision.append(round(float(prec), 2))
    ada_recall.append(round(float(re), 2))

    # Run metrics on Random Forest Classifier
    rf = RandomForestClassifier(random_state = 777)
    acc, prec, re = xval_classifier(rf, reduced_features, labels, sss)
    rf_accuracy.append(round(float(acc), 2))
    rf_precision.append(round(float(prec), 2))
    rf_recall.append(round(float(re), 2))
    print('Feature:', features_list[i])
    print('Time Spent Fitting k = {0}: {1}s'.format(i + 1, round(time() - t, 2)))
    print('AdaBoost Metrics: \nAccuracy: {0}, Precision: {1}, Recall: {2}'.format(ada_accuracy[-1], 
          ada_precision[-1], ada_recall[-1]))
    print('RandomForest Metrics: \nAccuracy: {0}, Precision: {1}, Recall: {2}'.format(rf_accuracy[-1], 
          rf_precision[-1], rf_recall[-1]))
    scores.append({'k': str(i + 1), 'feature': features_list[i], 'ada_accuracy': ada_accuracy[-1], 
                  'ada_precision': ada_precision[-1],  'ada_recall': ada_recall[-1], 
                  'rf_accuracy': rf_accuracy[-1], 'rf_precision': rf_precision[-1], 
                  'rf_recall': rf_recall[-1]})
    if i < len(features[0]):
        print('Please continue to wait...')
    line()

# Average scores from each feature to get an idea of how the classifiers compare overall
line()
print('Average Scores For Each Classifier:')

avg_ada_accuracy = [] 
avg_ada_precision = [] 
avg_ada_recall = []
avg_rf_accuracy = []
avg_rf_precision = []
avg_rf_recall = []

for k in scores:
    avg_ada_accuracy.append(k['ada_accuracy'])
    avg_ada_precision.append(k['ada_precision'])
    avg_ada_recall.append(k['ada_recall'])
    avg_rf_accuracy.append(k['rf_accuracy'])
    avg_rf_precision.append(k['rf_precision'])
    avg_rf_recall.append(k['rf_recall'])

print('Average AdaBoost Accuracy:', round(sum(avg_ada_accuracy) / len(avg_ada_accuracy), 2))
print('Average AdaBoost Precision:', round(sum(avg_ada_precision) / len(avg_ada_precision), 2))
print('Average AdaBoost Recall:', round(sum(avg_ada_recall) / len(avg_ada_recall), 2))
print('Average RandomForest Accuracy:', round(sum(avg_rf_accuracy) / len(avg_rf_accuracy), 2))
print('Average RandomForest Precision:', round(sum(avg_rf_precision) / len(avg_rf_precision), 2))
print('Average RandomForest Recall:', round(sum(avg_rf_recall) / len(avg_rf_recall), 2))
#line()
#print('Show score values for each feature')
#pprint.pprint(scores)

#Create plots to visualize what the scores of each feature look like
line()
print('Plot metrics for each classifier, showing the values of each k feature')
line()
ada_df = pd.DataFrame({'ada_accuracy': ada_accuracy, 'ada_precision': ada_precision, 
         'ada_recall': ada_recall})
rf_df = pd.DataFrame({'rf_accuracy': rf_accuracy, 'rf_precision': rf_precision, 
        'rf_recall': rf_recall})
ada_df.plot()
rf_df.plot()
#plt.show()
#Hide plots

print('When comparing AdaBoost to RandomForest, AdaBoost has a  higher average recall,')
print('and a more stable (but lower) precison.')
print('RandomForest has a higher average precision, with more variation, and a slightly ')
print('higher accuracy.')
print('It appears that overall RandomForest is doing a bit of a better job than AdaBoost.')

line()
# Use SelectKBest to score and identify the best features in each classifier
print('SelectKBest Features:')

max_ada_recall = ada_recall.index(max(ada_recall)) + 1
max_ada_precision = ada_precision.index(max(ada_precision)) + 1
max_rf_recall = rf_recall.index(max(rf_recall)) + 1
max_rf_precision = rf_precision.index(max(rf_precision)) + 1

bfl_ada_recall, bf_ada_recall = best_features(max_ada_recall, features, labels, features_list)
bfl_rf_recall, bf_rf_recall = best_features(max_rf_recall, features, labels, features_list)

print('Number of Ada Recall Best Features', len(bfl_ada_recall) - 1)
print('Number of RF Recall Best Features:', len(bfl_rf_recall) - 1)
print('AdaBoost has many more features that end up on the best features list than RandomForest')

line()
print('SelectKBest Features - Ada: {0}'.format(len(bfl_ada_recall) - 1))
for f in bfl_ada_recall[1:]:
    print(f + ' with a score of: {0}'.format(round(sel.scores_[bfl_ada_recall[1:].index(f)], 2)))
line()

RF = RandomForestClassifier(random_state = 777)
RF.fit(bf_rf_recall, labels)
print('RF Feature Importance:')
print(RF.feature_importances_)

print('Best Features - RF:')
for f in bfl_rf_recall[1:]:
    print(f + ' with a score of: {0}'.format(round(sel.scores_[bfl_rf_recall[1:].index(f)], 2)))
line()
print('With AdaBoost and RandomForest, there are 3 best features we can examine/tune.')
print('Precision for both classifiers is already over .3, however, RandomForest has a much')
print('lower average recall')

### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html
#rf_tuning_parameters = {'n_estimators': [90], 'min_samples_split': [3], 'max_features': [1]}
rf_tuning_parameters = {'n_estimators': [125], 
                        'min_samples_split': [2], 'max_features': [3]}
#ada_tuning_parametrers = {'n_estimators': [70], 'learning_rate': [.6]}
ada_tuning_parameters = {'n_estimators': [100],
                        'learning_rate': [1]}

ada_clf = tune_AdaBoost(ada_tuning_parameters, sss, bf_ada_recall, labels)
test_classifier(ada_clf, my_dataset, bfl_ada_recall, folds = 1000)
#print(xval_classifier(ada_clf, bf_ada_recall, labels, sss))


# Tune RandomForest
rf_clf = tune_RandomForest(rf_tuning_parameters, sss, bf_rf_recall, labels)
test_classifier(rf_clf, my_dataset, bfl_rf_recall, folds = 1000)
#print(xval_classifier(rf_clf, bf_rf_recall, labels, sss))

### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

#Dump Randomforest 
dump_classifier_and_data(rf_clf, my_dataset, bfl_rf_recall)

ImportError: No module named wk_functions